# Linear Regression with parameter

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('weight-height.csv')

In [3]:
df.head()

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801


In [4]:
df_dummies = pd.get_dummies(df[['Gender']])

In [5]:
dfTrasnformed = pd.concat([df[['Height', 'Weight']],
               df_dummies], axis=1)

In [6]:
dfTrasnformed.head()

,Height,Weight,Gender_Female,Gender_Male
0,73.847017,241.893563,0,1
1,68.781904,162.310473,0,1
2,74.110105,212.740856,0,1
3,71.730978,220.042470,0,1
4,69.881796,206.349801,0,1


In [7]:
X = dfTrasnformed[['Height','Gender_Female','Gender_Male']].values
y_true = dfTrasnformed['Weight'].values

In [8]:
X

array([[ 73.84701702,   0.        ,   1.        ],
       [ 68.78190405,   0.        ,   1.        ],
       [ 74.11010539,   0.        ,   1.        ],
       ..., 
       [ 63.86799221,   1.        ,   0.        ],
       [ 69.03424313,   1.        ,   0.        ],
       [ 61.94424588,   1.        ,   0.        ]])

In [9]:
y_true

array([ 241.89356318,  162.31047252,  212.74085556, ...,  128.47531878,
        163.85246135,  113.64910268])

## Linear Regression with Keras

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [11]:
model = Sequential()

In [12]:
model.add(Dense(1, input_shape=(3,)))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(Adam(lr=0.8), 'mean_squared_error')

In [15]:
model.fit(X, y_true, epochs=40)

Epoch 1/40
10000/10000 [==============================] - 0s - loss: 446.9186     
Epoch 2/40
10000/10000 [==============================] - 0s - loss: 205.3198     
Epoch 3/40
10000/10000 [==============================] - 0s - loss: 192.6033     
Epoch 4/40
10000/10000 [==============================] - 0s - loss: 189.1652     
Epoch 5/40
10000/10000 [==============================] - 0s - loss: 170.7300     
Epoch 6/40
10000/10000 [==============================] - 0s - loss: 160.7135     
Epoch 7/40
10000/10000 [==============================] - 0s - loss: 176.3297     
Epoch 8/40
10000/10000 [==============================] - 0s - loss: 140.3499     
Epoch 9/40
10000/10000 [==============================] - 0s - loss: 143.1845     
Epoch 10/40
10000/10000 [==============================] - 0s - loss: 130.8371     
Epoch 11/40
10000/10000 [==============================] - 0s - loss: 136.0255     
Epoch 12/40
10000/10000 [==============================] - 0s - loss: 122.8483     
E

In [16]:
y_pred = model.predict(X).ravel()

In [17]:
y_pred

array([ 216.54867554,  186.27526855,  218.12115479, ...,  137.82717896,
        168.70510864,  126.32923126], dtype=float32)

In [18]:
W, B = model.get_weights()

In [19]:
W

array([[   5.97685671],
       [-122.58493805],
       [-103.50661469]], dtype=float32)

In [20]:
B

array([-121.31771088], dtype=float32)

### Save Keras Model


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.save('HeightWeightExtended_model.h5')

### Export Model using CoreML Tools


In [23]:
from keras.models import load_model  
model = load_model('HeightWeightExtended_model.h5')  

In [24]:
import coremltools

In [27]:
coreml_model = coremltools.converters.keras.convert(model, input_names="matrixHeightFemaleMale", output_names="weight")

0 : dense_1_input, <keras.engine.topology.InputLayer object at 0x118443610>
1 : dense_1, <keras.layers.core.Dense object at 0x118443a50>


In [28]:
coreml_model.save('HeightWeightExtended_model.mlmodel')

### Test CoreML Model

In [29]:
coreml_model

input {
  name: "matrixHeightFemaleMale"
  type {
    multiArrayType {
      shape: 3
      dataType: DOUBLE
    }
  }
}
output {
  name: "weight"
  type {
    multiArrayType {
      shape: 1
      dataType: DOUBLE
    }
  }
}

In [36]:
coreml_model.predict({'matrixHeightFemaleMale' : [76.7717, 1, 0]})

NameError: name 'Female' is not defined

In [35]:
coreml_model.predict({'matrixHeightFemaleMale' : [76.7717, 0, 1]}) //Male

{u'weight': array([ 111.44416046])}